In [4]:
pip install pathlib

DEPRECATION: Loading egg at /home/blockchain/anaconda3/lib/python3.11/site-packages/mmfashion-0.4.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/blockchain/anaconda3/lib/python3.11/site-packages/torchvision-0.18.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [78]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [79]:
train_path = Path('/home/blockchain/Downloads/FIsh classification/images.cv_jzk6llhf18tm3k0kyttxz/data/train')
test_path = Path('/home/blockchain/Downloads/FIsh classification/images.cv_jzk6llhf18tm3k0kyttxz/data/test')
val_path = Path('/home/blockchain/Downloads/FIsh classification/images.cv_jzk6llhf18tm3k0kyttxz/data/val')

In [80]:
def create_dataframe(data_path):

    filepaths = list(data_path.glob(r'**/*.jpg'))
    labels = list(map(lambda x:os.path.split(os.path.split(x)[0])[1] , filepaths))

    filepaths = pd.Series(filepaths,name='Filepaths').astype(str)
    labels = pd.Series(labels,name='Lables')

    return pd.concat([filepaths,labels],axis=1)

In [81]:
train_df = create_dataframe(train_path)
test_df = create_dataframe(test_path)
val_df = create_dataframe(val_path)

In [82]:
train_df.head()

,Filepaths,Lables
0,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
1,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
2,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
3,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
4,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass


In [22]:
train_df

,Filepaths,Lables
0,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
1,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
2,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
3,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
4,/home/blockchain/Downloads/FIsh classification...,fish sea_food sea_bass
...,...,...
6220,/home/blockchain/Downloads/FIsh classification...,fish sea_food red_sea_bream
6221,/home/blockchain/Downloads/FIsh classification...,fish sea_food red_sea_bream
6222,/home/blockchain/Downloads/FIsh classification...,fish sea_food red_sea_bream
6223,/home/blockchain/Downloads/FIsh classification...,fish sea_food red_sea_bream


In [40]:
print(train_df['Lables'].nunique())

11


In [41]:
train_df['Lables'].value_counts()

Lables
animal fish                         1096
fish sea_food trout                  580
fish sea_food red_mullet             579
fish sea_food shrimp                 576
fish sea_food hourse_mackerel        573
fish sea_food red_sea_bream          571
fish sea_food black_sea_sprat        569
fish sea_food gilt_head_bream        566
fish sea_food striped_red_mullet     547
fish sea_food sea_bass               538
animal fish bass                      30
Name: count, dtype: int64

In [83]:
samples = []

for category in train_df['Lables'].unique():
    if category == "animal fish bass":
        samples.append(train_df.query("Lables == @category"))
    else:
        category_slice = train_df.query("Lables == @category")
        samples.append(category_slice.sample(200,random_state=1))


train_df =  pd.concat(samples,axis=0).sample(frac=1.0,random_state=1).reset_index(drop=True)


In [84]:
train_df['Lables'].value_counts()

Lables
fish sea_food sea_bass              200
fish sea_food trout                 200
fish sea_food gilt_head_bream       200
fish sea_food red_sea_bream         200
fish sea_food black_sea_sprat       200
animal fish                         200
fish sea_food hourse_mackerel       200
fish sea_food striped_red_mullet    200
fish sea_food red_mullet            200
fish sea_food shrimp                200
animal fish bass                     30
Name: count, dtype: int64

In [85]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,

)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    
)

In [86]:
train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepaths',
    y_col = 'Lables',
    target_size = (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed =42,
)

val_images = train_generator.flow_from_dataframe(
    dataframe = val_df,
    x_col = 'Filepaths',
    y_col = 'Lables',
    target_size = (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = True,
    seed =42,
)

test_images = test_generator.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'Filepaths',
    y_col = 'Lables',
    target_size = (224,224),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 32,
    shuffle = False
)

Found 2030 validated image filenames belonging to 11 classes.


Found 1092 validated image filenames belonging to 11 classes.
Found 3187 validated image filenames belonging to 11 classes.


In [87]:
mobilenet_model = tf.keras.applications.MobileNetV2(
    input_shape = (224,224,3),
    include_top = False,
    weights="imagenet",
    pooling = "avg",

)

mobilenet_model.trainable = False

In [ ]:
mobilenet_model.summary()

In [88]:
train_images.labels

[7,
 7,
 10,
 3,
 6,
 7,
 2,
 0,
 6,
 4,
 10,
 9,
 9,
 2,
 4,
 0,
 0,
 2,
 4,
 5,
 0,
 3,
 8,
 6,
 10,
 9,
 8,
 6,
 9,
 2,
 2,
 3,
 6,
 4,
 2,
 5,
 2,
 9,
 10,
 2,
 3,
 0,
 0,
 2,
 0,
 6,
 10,
 5,
 0,
 5,
 10,
 8,
 9,
 5,
 8,
 5,
 7,
 5,
 8,
 2,
 10,
 9,
 3,
 2,
 4,
 10,
 9,
 10,
 7,
 2,
 8,
 0,
 7,
 9,
 9,
 3,
 7,
 8,
 9,
 2,
 8,
 5,
 6,
 4,
 6,
 2,
 2,
 10,
 7,
 8,
 9,
 4,
 8,
 9,
 8,
 6,
 0,
 4,
 5,
 7,
 2,
 10,
 4,
 10,
 3,
 9,
 8,
 6,
 4,
 10,
 7,
 5,
 4,
 0,
 4,
 5,
 3,
 3,
 3,
 7,
 0,
 6,
 0,
 3,
 7,
 3,
 2,
 9,
 5,
 0,
 5,
 7,
 10,
 7,
 0,
 0,
 3,
 8,
 9,
 7,
 9,
 7,
 6,
 2,
 7,
 4,
 8,
 4,
 3,
 10,
 10,
 3,
 10,
 9,
 5,
 3,
 0,
 9,
 9,
 4,
 8,
 10,
 3,
 4,
 5,
 9,
 10,
 5,
 10,
 3,
 9,
 4,
 3,
 0,
 8,
 0,
 2,
 0,
 9,
 7,
 10,
 4,
 8,
 3,
 6,
 0,
 3,
 10,
 6,
 0,
 6,
 0,
 0,
 2,
 4,
 4,
 2,
 7,
 3,
 7,
 4,
 6,
 4,
 9,
 4,
 3,
 0,
 0,
 9,
 7,
 9,
 3,
 10,
 4,
 0,
 4,
 4,
 5,
 3,
 8,
 5,
 1,
 3,
 4,
 6,
 9,
 3,
 6,
 4,
 10,
 3,
 5,
 6,
 4,
 5,
 7,
 7,
 0,
 9,
 5,
 4,
 10,
 6,
 3,

In [91]:
inputs = mobilenet_model.input

x = tf.keras.layers.Dense(128,activation='relu')(mobilenet_model.output)
x = tf.keras.layers.Dense(128,activation='relu')(x)

outputs = tf.keras.layers.Dense(11,activation='softmax')(x)

model = tf.keras.Model(inputs = inputs,outputs = outputs)

model.compile(  
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

history = model.fit(
    train_images,
    validation_data = val_images,
    epochs =10,
)

Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 23s 305ms/step - accuracy: 0.5796 - loss: 1.4948 - val_accuracy: 0.9643 - val_loss: 0.1320
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 283ms/step - accuracy: 0.9795 - loss: 0.0826 - val_accuracy: 0.9780 - val_loss: 0.0730
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 287ms/step - accuracy: 0.9969 - loss: 0.0255 - val_accuracy: 0.9799 - val_loss: 0.0667
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 278ms/step - accuracy: 0.9988 - loss: 0.0149 - val_accuracy: 0.9835 - val_loss: 0.0481
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 17s 270ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 0.9844 - val_loss: 0.0444
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 288ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 0.9863 - val_loss: 0.0422
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 281ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 0.9890 - val_loss: 0.0377
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 18s 277ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accu

In [ ]:
result = model.evaluate(test_images,verbose = 0)
print(result)

[0.023985236883163452, 0.9927831888198853]
test loss:
test accuracy:


In [100]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

# Load and preprocess the image
def load_and_preprocess_image(img_path, target_size=(224, 224)):  # Adjust size based on your model
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize if needed
    return img_array

# Path to the image you want to test
img_path = "images.cv_jzk6llhf18tm3k0kyttxz/data/val/fish sea_food red_sea_bream/2KVXTG8MLUAH.jpg"

# Load and preprocess the image
img_array = load_and_preprocess_image(img_path)

# Get predictions
predictions = model.predict(img_array)

# Get the class with the highest probability
predicted_class = np.argmax(predictions, axis=1)[0]

# Print results
print(f"Predicted Class Index: {predicted_class}")

# If you have class names (labels), map the index to the label
class_names = sorted(train_df['Lables'].unique())  # Replace with actual class labels
print(f"Predicted Class Name: {class_names[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted Class Index: 6
Predicted Class Name: fish sea_food red_sea_bream
